In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
cd /raid/home/myang349/recsys-filterbubbles/

/raid/home/myang349/recsys-filterbubbles


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.optim as optim
from os import listdir
from os.path import isfile, join
from tracin.tracin import (
    save_tracin_checkpoint,
    load_tracin_checkpoint,
    calculate_tracin_influence,
)
import pandas as pd
from LSTM_clean.utils import filter_and_split_data, sequence_generator, printl
from LSTM_clean.model import LSTM 
from collections import Counter
import numpy as np
import pickle
from collections import defaultdict

In [5]:
os.getcwd()

'/raid/home/myang349/recsys-filterbubbles'

In [6]:
DATA_FOLDER = "/raid/home/myang349/recsys-filterbubbles/data/"
DATA_NAME = "twitch100k.csv"
USER_KEY = 'user_id'
ITEM_KEY = 'streamer_name'
TIME_KEY = 'stop_time'
LOOK_BACK = 50

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is ", device)


Device is  cuda


# Loading Data

In [8]:
print(f"Starting data retrieval for: {DATA_NAME}")
df = pd.read_csv(os.path.join(DATA_FOLDER, DATA_NAME))

Starting data retrieval for: twitch100k.csv


In [9]:
# Not needed because it is already done by the sequence generator function
# if 0 in df['streamer_name'].values:
#     df['streamer_name'] = df['streamer_name'].apply(lambda x: x + 1)

In [10]:
data = df[[USER_KEY, ITEM_KEY, TIME_KEY]]
data = data.values

In [15]:
unique_users = sorted(list(set(data[:, 0])))
unique_items = sorted(list(set(data[:, 1])))

In [16]:

user_dic = {user:idx for (idx,user) in enumerate(unique_users)}
item_dic = {item:idx for (idx,item) in enumerate(unique_items)}

# for (idx, row) in enumerate(data):
#     user,item,time = user_dic[row[0]],item_dic[row[1]],row[2]
#     data[idx,0],data[idx,1] = int(user),int(item)

In [13]:
latest_time_per_user

NameError: name 'latest_time_per_user' is not defined

In [29]:
new_data = filter_and_split_data(data)

In [31]:
new_data[:-10]

array([[   1,    0,  156,    0],
       [   1,    1,  169,    0],
       [   1,    0,  588,    0],
       ...,
       [4416,  468, 3166,    0],
       [4416, 2149, 3283,    0],
       [4416, 1984, 3291,    0]])

In [40]:
train[0][0].shape

torch.Size([49, 128])

In [42]:
train[1][0].shape

torch.Size([49, 128])

In [44]:
items_per_user = {int(user) : [0 for i in range(LOOK_BACK)] for user in unique_users}

# Train Test Split

In [22]:
data[0]

array([  1,   0, 156])

In [24]:
(users, counts) = np.unique(data[:, 0], return_counts=True)
users = users[counts >= 10]

In [25]:
print(len(users))


2529


In [28]:
print(len(np.unique(data[:, 0])))

4296


In [59]:
min_items_per_user = 10
train_cutoff = 0.8
valid_cutoff = 0.9
test_cutoff = 1.0

In [32]:
data

array([[   1,    0,  156],
       [   1,    1,  169],
       [   1,    0,  588],
       ...,
       [4416, 1515, 4015],
       [4416, 1984, 4008],
       [4416, 2375, 4022]])

In [42]:
users, counts = np.unique(data[:,0], return_counts=True)
for user, count in zip(users, counts):
    print('lol')
    break

lol


In [37]:
for user, count in np.unique(data[:, 0], return_counts=True):
    print(user, count)
    break

ValueError: too many values to unpack (expected 2)

In [67]:
    
# Only keep users with >= threshold items
users, counts = np.unique(data[:, 0], return_counts=True)
total_items_per_user = {user: count for user, count in zip(users, counts) if count >= min_items_per_user}
count_per_user = defaultdict(int)

new_data = []
for user, item, time in data:
    if user not in total_items_per_user:
        continue
    else:
        total_items = total_items_per_user[user]
        
    count_per_user[user] += 1
    
    if count_per_user[user] <= train_cutoff * total_items:
        new_data.append([user, item, time, 0])
    elif count_per_user[user] <= valid_cutoff * total_items:
        new_data.append([user, item, time, 1])
    elif count_per_user[user] <= total_items:
        new_data.append([user, item, time, 2])
    else:
        raise ValueError("There's a bug in the counting of total items or curr items!")

In [45]:
# from dataclasses import dataclass

# @dataclass
# class Item:
#     item: int
#     time: int
    
# # Only keep users with >= threshold items
# users, counts = np.unique(data[:, 0], return_counts=True)
# total_items_per_user = {user: count for user, count in zip(users, counts) if count >= min_items_per_user}
# count_per_user = defaultdict(int)

# new_data = []
# for user, item, time in data:
#     if user not in total_items_per_user:
#         continue
#     else:
#         total_items = total_items_per_user[user]
        
#     count_per_user[user] += 1
    
#     train_cutoff = train_ratio * (1 - valid_ratio) * total_items
    
#     # Strictly training
#     if count_per_user[user] < train_ratio * (1 - valid_ratio) * total_items:
#         new_data.append([user, item, time, 0])
#     # Validation data
#     elif count_per_user[user] < train_ratio * total_items:
#         new_data.append([user, item, time, 1])
#     # Testing
#     else:
#         new_data.append([user, item, time, 2])
        
    
    
    
        
        
    
    
# # new_data = 
# # for i in data.shape[0]:
# #     user, item, time = data[i]
    
    

# # sequence_dictt = {int(user): [] for user in set(data[:, 0])}
# sequence_dict = defaultdict(list)

# # sequence_dict, pert_dic = {int(user): [] for user in set(data[:, 0])}, {
# #     int(user): [] for user in set(data[:, 0])
# # }


# # user_dic = {int(user): idx for (idx, user) in enumerate(users)}

# # new_data = []
# # for row in range(len(data)):
# #     user, item, time = data[i]

# #     if user in user_dic:
# #         new_data.append([user, item, time, 0])

# # new_data = np.array(new_data)

# # for i in range(new_data.shape[0]):
# #     user, item, time = new_data[i]
# #     sequence_dict[user].append([i, item, time])

# # for user in sequence_dict.keys():
# #     cur_test = int(0.05 * len(sequence_dict[user]))
# #     for i in range(cur_test):
# #         interaction = sequence_dict[user].pop()
# #         new_data[interaction[0], 3] = 2

#     # cur_val = int(0.1*len(sequence_dict[user]))
#     # for i in range(cur_val):
#     #    interaction = sequence_dict[user].pop()
#     #    new_data[interaction[0],3] = 1

In [48]:
max(num_items_per_user.values())

209

# Continued Data Processing

In [46]:
# In[5] Data preprocessing
print("Preprocessing Data")
original_data = train_test_split(data=data)
(train,valid, test) = sequence_generator(original_data,LOOK_BACK)
test_ground_truth = {i:test[i][1] for i in range(len(test))}

print("Train: {}, Test: {}".format(len(train),len(test)))



# # %%
# # In[6] Run LSTM
# model = LSTM(input_size=128, output_size=len(unique_items)+1, hidden_dim=64, n_layers=1, device=device).to(device)
# model.LSTM.flatten_parameters()
# print("Model is ", model)
# print("Training and testing")
# original_prediction = model.traintest(train=train,test=test,epochs = 500)
# print("Finished")

Preprocessing Data


In [65]:
with open(f"{DATA_FOLDER}sequence/temp.data", 'wb') as f:
    pickle.dump(train, f)

In [67]:
with open(f"{DATA_FOLDER}sequence/temp.data", "rb") as f:
    data = pickle.load(f)

In [70]:
os.path.getsize(f"{DATA_FOLDER}sequence/temp.data")

11056373

# junk below

In [ ]:
assert torch.cuda.is_available()
device = torch.device("cuda")
print(f"Device is: {device}")

In [28]:
df = pd.read_csv(os.path.join(DATA_FOLDER, DATA))

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is ", device)


# In[4] Load data (ex: wikipedia [user, item, timestamp])
print("Loading data")
path = os.getcwd()
raw_data = pd.read_csv(path + "/data/twitch100k.csv", sep=',', header=0)
print(raw_data.head())
# streamer_community_dict = pd.Series(raw_data['community'].values, index=raw_data['streamer_name']).to_dict()
data = raw_data[['user_id', 'streamer_name', 'stop_time']]
print(data.head())
data = data.values
look_back = 50
print(data)

Device is  cuda
Loading data
   user_id    stream_id  streamer_name  start_time  stop_time  community
0        1  33842865744              0         154        156          5
1        1  33846768288              1         166        169          5
2        1  33886469056              0         587        588          5
3        1  33887624992              2         589        591          5
4        1  33890145056              3         591        594          5
   user_id  streamer_name  stop_time
0        1              0        156
1        1              1        169
2        1              0        588
3        1              2        591
4        1              3        594
[[   1    0  156]
 [   1    1  169]
 [   1    0  588]
 ...
 [4416 1515 4015]
 [4416 1984 4008]
 [4416 2375 4022]]


In [73]:
len(set(df['streamer_name']))

5672

In [72]:
max(df['streamer_name'])

5671

In [9]:
nums = [1,2,3]

In [10]:
x, y, z = nums

In [11]:
x

1